# Openpose light weight Test
 - https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch

# lightweight-human-pose-estimation.pytorch Setting

In [20]:
%cd /content/

import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/Daniil-Osokin/lightweight-human-pose-estimation.pytorch.git'
git_path = '/content/lightweight-human-pose-estimation.pytorch'
pre_model_path = os.path.join(git_path,'pre_model')

project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  !git clone -q --depth 1 $git_repo_url
  !wget "http://images.cocodataset.org/annotations/annotations_trainval2017.zip" -P $git_path
  !wget "https://download.01.org/opencv/openvino_training_extensions/models/human_pose_estimation/checkpoint_iter_370000.pth" -P $pre_model_path
  !mkdir coco && unzip "/content/lightweight-human-pose-estimation.pytorch/annotations_trainval2017.zip" -d "/content/coco/"

%cd $git_path

/content
/content/lightweight-human-pose-estimation.pytorch


In [23]:
!pip install -r requirements.txt

# Demo - mp4
 - colab ( demo_test.py )

In [24]:
!rm -f demo_test.py

## create demo_test.py

In [25]:
%%writefile demo_test.py
import argparse

import cv2
import numpy as np
import torch

from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
from val import normalize, pad_width


class ImageReader(object):
    def __init__(self, file_names):
        self.file_names = file_names
        self.max_idx = len(file_names)

    def __iter__(self):
        self.idx = 0
        return self

    def __next__(self):
        if self.idx == self.max_idx:
            raise StopIteration
        img = cv2.imread(self.file_names[self.idx], cv2.IMREAD_COLOR)
        if img.size == 0:
            raise IOError('Image {} cannot be read'.format(self.file_names[self.idx]))
        self.idx = self.idx + 1
        return img


class VideoReader(object):
    def __init__(self, file_name):
        self.file_name = file_name
        try:  # OpenCV needs int to read from webcam
            self.file_name = int(file_name)
        except ValueError:
            pass

    def __iter__(self):
        self.cap = cv2.VideoCapture(self.file_name)
        if not self.cap.isOpened():
            raise IOError('Video {} cannot be opened'.format(self.file_name))
        return self

    def __next__(self):
        was_read, img = self.cap.read()
        if not was_read:
            raise StopIteration
        return img


def infer_fast(net, img, net_input_height_size, stride, upsample_ratio, cpu,
               pad_value=(0, 0, 0), img_mean=(128, 128, 128), img_scale=1/256):
    height, width, _ = img.shape
    scale = net_input_height_size / height

    scaled_img = cv2.resize(img, (0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_CUBIC)
    scaled_img = normalize(scaled_img, img_mean, img_scale)
    min_dims = [net_input_height_size, max(scaled_img.shape[1], net_input_height_size)]
    padded_img, pad = pad_width(scaled_img, stride, pad_value, min_dims)

    tensor_img = torch.from_numpy(padded_img).permute(2, 0, 1).unsqueeze(0).float()
    if not cpu:
        tensor_img = tensor_img.cuda()

    stages_output = net(tensor_img)

    stage2_heatmaps = stages_output[-2]
    heatmaps = np.transpose(stage2_heatmaps.squeeze().cpu().data.numpy(), (1, 2, 0))
    heatmaps = cv2.resize(heatmaps, (0, 0), fx=upsample_ratio, fy=upsample_ratio, interpolation=cv2.INTER_CUBIC)

    stage2_pafs = stages_output[-1]
    pafs = np.transpose(stage2_pafs.squeeze().cpu().data.numpy(), (1, 2, 0))
    pafs = cv2.resize(pafs, (0, 0), fx=upsample_ratio, fy=upsample_ratio, interpolation=cv2.INTER_CUBIC)

    return heatmaps, pafs, scale, pad


def run_demo(net, image_provider, height_size, cpu, track, smooth):
    net = net.eval()
    if not cpu:
        net = net.cuda()

    stride = 8
    upsample_ratio = 4
    num_keypoints = Pose.num_kpts
    previous_poses = []
    delay = 33

    frame = image_provider.__iter__().__next__()
    vid_writer = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (frame.shape[1],frame.shape[0]))

    for img in image_provider:
        orig_img = img.copy()
        heatmaps, pafs, scale, pad = infer_fast(net, img, height_size, stride, upsample_ratio, cpu)

        total_keypoints_num = 0
        all_keypoints_by_type = []
        for kpt_idx in range(num_keypoints):  # 19th for bg
            total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

        pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs, demo=True)
        for kpt_id in range(all_keypoints.shape[0]):
            all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
            all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
        current_poses = []
        for n in range(len(pose_entries)):
            if len(pose_entries[n]) == 0:
                continue
            pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
            for kpt_id in range(num_keypoints):
                if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
                    pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
                    pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
            pose = Pose(pose_keypoints, pose_entries[n][18])
            current_poses.append(pose)

        if track:
            track_poses(previous_poses, current_poses, smooth=smooth)
            previous_poses = current_poses
        for pose in current_poses:
            pose.draw(img)
        img = cv2.addWeighted(orig_img, 0.6, img, 0.4, 0)
        for pose in current_poses:
            cv2.rectangle(img, (pose.bbox[0], pose.bbox[1]),
                          (pose.bbox[0] + pose.bbox[2], pose.bbox[1] + pose.bbox[3]), (0, 255, 0))
            if track:
                cv2.putText(img, 'id: {}'.format(pose.id), (pose.bbox[0], pose.bbox[1] - 16),
                            cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))
        #cv2.imshow('Lightweight Human Pose Estimation Python Demo', img)
        vid_writer.write(img)
        key = cv2.waitKey(delay)
        if key == 27:  # esc
            return
        elif key == 112:  # 'p'
            if delay == 33:
                delay = 0
            else:
                delay = 33
    vid_writer.release()

if __name__ == '__main__':
    parser = argparse.ArgumentParser(
        description='''Lightweight human pose estimation python demo.
                       This is just for quick results preview.
                       Please, consider c++ demo for the best performance.''')
    parser.add_argument('--checkpoint-path', type=str, required=True, help='path to the checkpoint')
    parser.add_argument('--height-size', type=int, default=256, help='network input layer height size')
    parser.add_argument('--video', type=str, default='', help='path to video file or camera id')
    parser.add_argument('--images', nargs='+', default='', help='path to input image(s)')
    parser.add_argument('--cpu', action='store_true', help='run network inference on cpu')
    parser.add_argument('--track', type=int, default=1, help='track pose id in video')
    parser.add_argument('--smooth', type=int, default=1, help='smooth pose keypoints')
    args = parser.parse_args()

    if args.video == '' and args.images == '':
        raise ValueError('Either --video or --image has to be provided')

    net = PoseEstimationWithMobileNet()
    checkpoint = torch.load(args.checkpoint_path, map_location='cpu')
    load_state(net, checkpoint)

    frame_provider = ImageReader(args.images)
    if args.video != '':
        frame_provider = VideoReader(args.video)
    else:
        args.track = 0

    run_demo(net, frame_provider, args.height_size, args.cpu, args.track, args.smooth)


Writing demo_test.py


## run demo_test.py
 - upload video
 - run

In [26]:
%%shell
rm -rf videos
mkdir videos; cd videos
curl -LJO https://github.com/jeeenn/SportsFriends/raw/master/Data/yogaFriend_demo.mp4  > test.mp4
cd videos
mkdir save

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2821k  100 2821k    0     0  2396k      0  0:00:01  0:00:01 --:--:-- 62.6M
/bin/bash: line 4: cd: videos: No such file or directory


In [27]:
# GPU
!python demo_test.py --checkpoint-path $pre_model_path/checkpoint_iter_370000.pth --video '/content/lightweight-human-pose-estimation.pytorch/videos/yogaFriend_demo.mp4'
# CPU
#!python demo_test.py --checkpoint-path $pre_model_path/checkpoint_iter_370000.pth --cpu --video '/content/lightweight-human-pose-estimation.pytorch/videos/yogaFriend_demo.mp4'

/content/lightweight-human-pose-estimation.pytorch/demo_test.py:162: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.checkpoint_path, map_location

In [28]:
!ffmpeg -y -loglevel info -i 'output.avi' openpose.mp4

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [29]:
import io
import base64
from IPython.display import HTML
file_name = "openpose.mp4"
video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                      <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                    </video>'''.format(640, 480, video_encoded.decode('ascii')))

# Real Time Pose Detection
  - colab ( ing )

In [30]:
import base64, logging
import numpy as np
from PIL import Image
from io import BytesIO

def data_uri_to_img(uri):
  try:
    image = base64.b64decode(uri.split(',')[1], validate=True)
    # make the binary image, a PIL image
    image = Image.open(BytesIO(image))
    # convert to numpy array
    image = np.array(image, dtype=np.uint8);
    return image
  except Exception as e:
    logging.exception(e);print('\n')
    return None

def video_to_data_url(filename):
    ext = filename.split('.')[-1]
    prefix = 'data:video/{};base64,'.format(ext)
    with open(filename, 'rb') as f:
        vidoe = f.read()
    return prefix + base64.b64encode(vidoe).decode()


In [31]:
import cv2 as cv
import torch

from models.with_mobilenet import PoseEstimationWithMobileNet
from modules.keypoints import extract_keypoints, group_keypoints
from modules.load_state import load_state
from modules.pose import Pose, track_poses
from val import normalize, pad_width

def infer_fast(net, img, net_input_height_size, stride, upsample_ratio, cpu,
               pad_value=(0, 0, 0), img_mean=(128, 128, 128), img_scale=1/256):
    height, width, _ = img.shape
    scale = net_input_height_size / height

    scaled_img = cv.resize(img, (0, 0), fx=scale, fy=scale, interpolation=cv.INTER_CUBIC)
    scaled_img = normalize(scaled_img, img_mean, img_scale)
    min_dims = [net_input_height_size, max(scaled_img.shape[1], net_input_height_size)]
    padded_img, pad = pad_width(scaled_img, stride, pad_value, min_dims)

    tensor_img = torch.from_numpy(padded_img).permute(2, 0, 1).unsqueeze(0).float()
    if not cpu:
        tensor_img = tensor_img.cuda()

    stages_output = net(tensor_img)

    stage2_heatmaps = stages_output[-2]
    heatmaps = np.transpose(stage2_heatmaps.squeeze().cpu().data.numpy(), (1, 2, 0))
    heatmaps = cv2.resize(heatmaps, (0, 0), fx=upsample_ratio, fy=upsample_ratio, interpolation=cv2.INTER_CUBIC)

    stage2_pafs = stages_output[-1]
    pafs = np.transpose(stage2_pafs.squeeze().cpu().data.numpy(), (1, 2, 0))
    pafs = cv2.resize(pafs, (0, 0), fx=upsample_ratio, fy=upsample_ratio, interpolation=cv2.INTER_CUBIC)

    return heatmaps, pafs, scale, pad


In [32]:
def detect_key_point(frame):
  #(frameWidth, frameHeight) = frame.shape[:2]
  #inp = cv.dnn.blobFromImage(frame, inScale, (inWidth, inHeight),
  #                            (0, 0, 0), swapRB=False, crop=False)

  model_path = os.path.join(pre_model_path,'checkpoint_iter_370000.pth')
  height_size = 1
  cpu = True

  net = PoseEstimationWithMobileNet()
  checkpoint = torch.load(model_path, map_location='cpu')
  load_state(net, checkpoint)
  #net = cv.dnn.readNetFromModelOptimizer(protoFile, weightsFile)
  net = net.eval()
  if not cpu:
      net = net.cuda()

  stride = 8
  upsample_ratio = 4
  num_keypoints = Pose.num_kpts
  previous_poses = []
  delay = 33
  orig_img = img.copy()
  heatmaps, pafs, scale, pad = infer_fast(net, frame, height_size, stride, upsample_ratio, cpu)

  total_keypoints_num = 0
  all_keypoints_by_type = []
  for kpt_idx in range(num_keypoints):  # 19th for bg
      total_keypoints_num += extract_keypoints(heatmaps[:, :, kpt_idx], all_keypoints_by_type, total_keypoints_num)

  pose_entries, all_keypoints = group_keypoints(all_keypoints_by_type, pafs, demo=True)
  for kpt_id in range(all_keypoints.shape[0]):
      all_keypoints[kpt_id, 0] = (all_keypoints[kpt_id, 0] * stride / upsample_ratio - pad[1]) / scale
      all_keypoints[kpt_id, 1] = (all_keypoints[kpt_id, 1] * stride / upsample_ratio - pad[0]) / scale
  current_poses = []
  for n in range(len(pose_entries)):
      if len(pose_entries[n]) == 0:
          continue
      pose_keypoints = np.ones((num_keypoints, 2), dtype=np.int32) * -1
      for kpt_id in range(num_keypoints):
          if pose_entries[n][kpt_id] != -1.0:  # keypoint was found
              pose_keypoints[kpt_id, 0] = int(all_keypoints[int(pose_entries[n][kpt_id]), 0])
              pose_keypoints[kpt_id, 1] = int(all_keypoints[int(pose_entries[n][kpt_id]), 1])
      pose = Pose(pose_keypoints, pose_entries[n][18])
      current_poses.append(pose)

      if track:
          track_poses(previous_poses, current_poses, smooth=smooth)
          previous_poses = current_poses
      for pose in current_poses:
          pose.draw(img)
      img = cv.addWeighted(orig_img, 0.6, img, 0.4, 0)
      for pose in current_poses:
          cv.rectangle(img, (pose.bbox[0], pose.bbox[1]),
                        (pose.bbox[0] + pose.bbox[2], pose.bbox[1] + pose.bbox[3]), (0, 255, 0))
          if track:
              cv.putText(img, 'id: {}'.format(pose.id), (pose.bbox[0], pose.bbox[1] - 16),
                          cv.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 255))


In [33]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();

      for (let i = 0; isOpened; i++) {
        canvas.getContext("2d").clearRect(0, 0, canvas.width, canvas.height);
        canvas.getContext('2d').drawImage(video, 0, 0);
        img = canvas.toDataURL('image/jpeg', quality);

        // jsLog(i + "sending");
        // Call a python function and send this image
        google.colab.kernel.invokeFunction('notebook.run_PoseDetection', [img], {});
        // jsLog(i + "SENT");

        // wait for X miliseconds second, before next capture
        await new Promise(resolve => setTimeout(resolve, 250));
      }

      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [34]:
from google.colab import output
frame_count = 0
writer = None

# InvokeFunction
# takes the numpy image and runs detection, then shows the results by visualizing
def run_PoseDetection(uri):
  global frame_count, writer

  image = data_uri_to_img(uri)
  if writer is None:
      fourcc = cv.VideoWriter_fourcc(*'DIVX')
      writer = cv.VideoWriter(outVideo, fourcc, 2, (image.shape[1], image.shape[0]), True)
  try:
    detect_key_point(image)

    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
    frame_count+=1
    name = '{0}.jpg'.format(frame_count)
    name = os.path.join(VIDEO_SAVE_DIR, name)
    cv.imwrite(name, image)

    if writer is not None:
      writer.write(image)
  except Exception as e:
    logging.exception(e)
    print('\n')

# register this function, so JS code could call this
output.register_callback('notebook.run_PoseDetection', run_PoseDetection)

In [35]:
import os

VIDEO_DIR = os.path.join(git_path, "videos")
VIDEO_SAVE_DIR = os.path.join(VIDEO_DIR, "save")

In [36]:
inVideo = os.path.join(VIDEO_DIR, "test.mp4")
outVideo= os.path.join(VIDEO_DIR, "out.avi")

In [37]:
%%shell
mkdir videos; cd videos
curl -LJO https://github.com/jeeenn/SportsFriends/raw/master/Data/yogaFriend_demo.mp4  > test.mp4
cd videos
mkdir save

mkdir: cannot create directory ‘videos’: File exists
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 2821k  100 2821k    0     0  15.7M      0 --:--:-- --:--:-- --:--:-- 15.7M
/bin/bash: line 3: cd: videos: No such file or directory
mkdir: cannot create directory ‘save’: File exists


CalledProcessError: Command 'mkdir videos; cd videos
curl -LJO https://github.com/jeeenn/SportsFriends/raw/master/Data/yogaFriend_demo.mp4  > test.mp4
cd videos
mkdir save
' returned non-zero exit status 1.

In [38]:
frame_count = 0
data_url = video_to_data_url(inVideo)
try:
  # put the JS code in cell and run it
  take_photo(data_url)
  if writer is not None:
    writer.release()
except Exception as e:
  logging.exception(e)
  print('\n')

writer = None

<IPython.core.display.Javascript object>

ERROR:root:ReferenceError: isOpened is not defined
Traceback (most recent call last):
  File "<ipython-input-38-8521a4e8e319>", line 5, in <cell line: 3>
    take_photo(data_url)
  File "<ipython-input-33-0bf0092f58e0>", line 53, in take_photo
    data = eval_js('takePhoto({})'.format(quality))
  File "/usr/local/lib/python3.10/dist-packages/google/colab/output/_js.py", line 40, in eval_js
    return _message.read_reply_from_input(request_id, timeout_sec)
  File "/usr/local/lib/python3.10/dist-packages/google/colab/_message.py", line 103, in read_reply_from_input
    raise MessageError(reply['error'])
google.colab._message.MessageError: ReferenceError: isOpened is not defined


In [46]:
import cv2 as cv

In [47]:
import matplotlib.pyplot as plt
from models.with_mobilenet import PoseEstimationWithMobileNet
from datasets.transformations import Scale, Rotate, Flip, CropPad, ConvertKeypoints


import os

model_path = "/content/lightweight-human-pose-estimation.pytorch/models/with_mobilenet.py"

def load_model(model_path):
    load_model("/content/lightweight-human-pose-estimation.pytorch/models/with_mobilenet.py")
    # Load the pre-trained model
    model = pose_hrnet.get_pose_net()
    checkpoint = torch.load('/content/lightweight-human-pose-estimation.pytorch/models/with_mobilenet.py')
    model.load_state_dict(checkpoint['state_dict'])
    model.eval()
    return model

image_path = "/content/WhatsApp Image 2024-10-20 at 17.19.49_7b8c66cd (2).jpg"

def preprocess_image(image_path):
    # Read the image and convert it to RGB
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Normalize and resize the image
    image = cv2.resize(image, (256, 192))  # Default input size for HRNet
    image = image.astype(np.float32) / 255.0
    return image

def postprocess_output(output, image_shape):
    # Assuming output contains keypoint locations and heatmaps
    preds, _ = get_final_preds(output, center=None, scale=None)

    # Scale the predictions back to the original image dimensions
    preds = preds * np.array([image_shape[1], image_shape[0]])  # Convert to image space
    return preds

def plot_pose(image, keypoints):
    # Visualize the pose estimation
    plt.imshow(image)
    for point in keypoints:
        x, y = point
        plt.scatter(x, y, color='red', s=10)
    plt.show()

def apply_pose_estimation(image_path, model_path):
    # Load the image and preprocess it
    image = preprocess_image(image_path)

    # Load the model
    model = load_model(model_path)

    # Convert image to tensor and add batch dimension
    input_image = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0)

    # Run inference on the image
    with torch.no_grad():
        output = model(input_image)

    # Postprocess the output
    keypoints = postprocess_output(output, image.shape)

    # Plot the keypoints on the image
    plot_pose(image, keypoints)

if __name__ == "__main__":
    image_path = "path_to_your_image.jpg"  # Replace with your image path
    model_path = "path_to_your_pretrained_model.pth"  # Replace with the path to the pretrained model
    apply_pose_estimation(image_path, model_path)

NameError: name 'cv2' is not defined